Este notebook tiene la finalidad de realizar la carga, transformación y un posterior análisis (ETL + EDA) de los datasets del sitio Google Maps.

In [2]:
# Importamos las librerias a utilizar
import pandas as pd
import numpy as np
import os

review-estados

In [6]:
# Lee el archivo JSON y carga los datos en un DataFrame de pandas
df = pd.read_json("Datasets/Google Maps/review-estados/review-Alabama/11.json", lines = True)

# Muestra las primeras filas del DataFrame para verificar que se haya cargado correctamente
print(df.head())

        user_id                    name           time  rating  text  pics  \
0  1.116430e+20  Chris and Shea Tolbert  1542773927242       5  None  None   
1  1.179291e+20            clint walker  1549501934566       3  None  None   
2  1.130570e+20                Beth Box  1545440036434       1  None  None   
3  1.051583e+20            Todd Beasley  1530583141411       5  None  None   
4  1.139799e+20           brandi hunter  1483573068099       4  None  None   

   resp                                gmap_id  
0  None  0x88882a58884a0507:0x3bdeae3983aec4dc  
1  None  0x88882a58884a0507:0x3bdeae3983aec4dc  
2  None  0x88882a58884a0507:0x3bdeae3983aec4dc  
3  None  0x88882a58884a0507:0x3bdeae3983aec4dc  
4  None  0x88882a58884a0507:0x3bdeae3983aec4dc  


c:\Users\Mauricio Figueroa\.virtualenvs\Python-env--sma9hmZ\Lib\site-packages\pandas\core\dtypes\astype.py:189: RuntimeWarning: invalid value encountered in cast
  return values.astype(dtype, copy=copy)


In [7]:
# Cada archivo tiene aproximadamente 150 mil registros
len(df)

150000

In [23]:
# La información de cada review esta contenida en las siguientes columnas
df.columns

Index(['user_id', 'name', 'time', 'rating', 'text', 'pics', 'resp', 'gmap_id'], dtype='object')

In [30]:
# Vemos que hay algunos registros (pocos) donde el comercio hace una devolución. Consideramos importante esta devolución
df[~df["resp"].isna()]

,user_id,name,time,rating,text,pics,resp,gmap_id
60,1.019555e+20,MrNiceGuy420,1620252981528,5,"It's quiet , cameras everywhere and a hardee's...",None,"{'time': 1620253626918, 'text': 'Thank you for...",0x88882af73c1dd91f:0x399a5e0c64165330
73,1.061407e+20,Kenneth Smith,1573595680762,5,Awesome and safe place to do your laundry.clea...,None,"{'time': 1573836762445, 'text': 'Thank you! We...",0x88882af73c1dd91f:0x399a5e0c64165330
77,1.183360e+20,Brandy Gipson,1576542079720,5,Its very clean and the washing machines and dr...,None,"{'time': 1576608216452, 'text': 'Thank you Bra...",0x88882af73c1dd91f:0x399a5e0c64165330
97,1.107800e+20,Damien Long,1617727974641,4,👍🏼,None,"{'time': 1617732842952, 'text': 'Thank you for...",0x88882af73c1dd91f:0x399a5e0c64165330
103,1.145796e+20,Kelly Ballam,1616568755311,5,None,None,"{'time': 1617732881857, 'text': 'Thank you for...",0x88882af73c1dd91f:0x399a5e0c64165330
...,...,...,...,...,...,...,...,...
149919,1.086057e+20,Samantha Vines,1556313173819,5,None,None,"{'time': 1556826886268, 'text': 'Thanks so muc...",0x8889cbb390b6d7ab:0x62b42d79d9d55109
149920,1.046784e+20,Carolyn Pearl,1544271470421,5,None,None,"{'time': 1544480389291, 'text': 'Thanks Caroly...",0x8889cbb390b6d7ab:0x62b42d79d9d55109
149921,1.005816e+20,Madison Espy,1527015204203,5,None,None,"{'time': 1527106474830, 'text': 'Thanks-a-mill...",0x8889cbb390b6d7ab:0x62b42d79d9d55109
149926,1.145077e+20,Dale Blakely,1562189862455,5,None,None,"{'time': 1563993083330, 'text': 'Thanks so muc...",0x8889cbb390b6d7ab:0x62b42d79d9d55109


Hacemos una función genérica para recorrer todos los archivos de cada estado y formar un solo DataFrame con la info relevante

In [8]:
def cargar_archivos_json(ruta_base, ruta_destino):

    # Diccionario para llevar el conteo de reviews nulas (review o devolución) por estado
    conteo_nulos = {}

    # Recorre todas las carpetas dentro de la ruta_base
    for estado_folder in os.listdir(ruta_base):

        # Lista para almacenar los df que corresponden a cada json
        lista_dataframes = []

        estado_path = os.path.join(ruta_base, estado_folder)

        # Variable para guardar el nombre del estado
        estado_string = estado_folder.replace("review-", "")
        
        # Inicializa el conteo de nulos para el estado actual
        conteo_nulos[estado_string] = 0

        # Verifica si es una carpeta
        if os.path.isdir(estado_path):
            # Recorre todos los archivos JSON dentro de la carpeta del estado
            for json_file in os.listdir(estado_path):
                if json_file.endswith('.json'):
                    json_path = os.path.join(estado_path, json_file)

                    # Lee el archivo JSON y carga los datos en un DataFrame
                    df = pd.read_json(json_path, lines=True)

                    # Incrementa el conteo de nulos si "text" o "resp" son NaN
                    conteo_nulos[estado_string] += df[["text", "resp"]].isna().all(axis=1).sum()

                    # Limpiamos los nan en la columna de reviews en caso de que la review o la respuesta sean NaN
                    df = df.dropna(subset = ["text", "resp"], how = "all")

                    # Dejamos las columnas que nos interesan
                    df = df[["rating", "text", "resp", "gmap_id"]]

                    # Agrega el DataFrame a la lista
                    lista_dataframes.append(df)

        # Concatena los dataFrames en uno solo
        review_estado = pd.concat(lista_dataframes, ignore_index = True)

        # Exporta reviews_estado a un archivo CSV
        csv_path = os.path.join(ruta_destino, f"review-{estado_string}.csv")
        review_estado.to_csv(csv_path, index=False)

    return conteo_nulos

In [ ]:
# Ruta base donde se encuentran las carpetas "review-estados"
ruta_base = "Datasets/Google Maps/review-estados"
ruta_destino = "Datasets/Google Maps/review-estados-clean"

# Llama a la función y almacena el conteo de nulos
conteo_nulos = cargar_archivos_json(ruta_base, ruta_destino)

In [11]:
# Muestra el conteo de nulos por estado, si las columnas text y resp eran nulas
for estado, conteo in conteo_nulos.items():
    print(f"Estado: {estado}, Registros Nulos: {conteo}")

Estado: Alabama, Registros Nulos: 760061
Estado: Alaska, Registros Nulos: 208566
Estado: Arizona, Registros Nulos: 632210
Estado: Arkansas, Registros Nulos: 1011648
Estado: California, Registros Nulos: 1093815
Estado: Colorado, Registros Nulos: 807096
Estado: Connecticut, Registros Nulos: 187694
Estado: Delaware, Registros Nulos: 373253
Estado: District_of_Columbia, Registros Nulos: 255029
Estado: Florida, Registros Nulos: 957433
Estado: Georgia, Registros Nulos: 715159
Estado: Hawaii, Registros Nulos: 615152
Estado: Idaho, Registros Nulos: 795796
Estado: Illinois, Registros Nulos: 829685
Estado: Indiana, Registros Nulos: 912398
Estado: Iowa, Registros Nulos: 1190817
Estado: Kansas, Registros Nulos: 813840
Estado: Kentucky, Registros Nulos: 686014
Estado: Louisiana, Registros Nulos: 651816
Estado: Maine, Registros Nulos: 476105
Estado: Maryland, Registros Nulos: 972803
Estado: Massachusetts, Registros Nulos: 1052584
Estado: Michigan, Registros Nulos: 850459
Estado: Minnesota, Registros

In [18]:
# Registros nulos total
print(f"Registros Nulos total: {sum(conteo_nulos.values())}")

Registros Nulos total: 34613269


metadata-sitios

In [4]:
# Lee el archivo JSON y carga los datos en un DataFrame de pandas
df = pd.read_json("Datasets/Google Maps/metadata-sitios/1.json", lines = True)

In [6]:
df.head(2)

,name,address,gmap_id,description,latitude,longitude,category,avg_rating,num_of_reviews,price,hours,MISC,state,relative_results,url
0,Porter Pharmacy,"Porter Pharmacy, 129 N Second St, Cochran, GA ...",0x88f16e41928ff687:0x883dad4fd048e8f8,None,32.388300,-83.35710,[Pharmacy],4.9,16,None,"[[Friday, 8AM–6PM], [Saturday, 8AM–12PM], [Sun...","{'Service options': ['In-store shopping', 'Sam...",Open ⋅ Closes 6PM,"[0x88f16e41929435cf:0x5b2532a2885e9ef6, 0x88f1...",https://www.google.com/maps/place//data=!4m2!3...
1,City Textile,"City Textile, 3001 E Pico Blvd, Los Angeles, C...",0x80c2c98c0e3c16fd:0x29ec8a728764fdf9,None,34.018891,-118.21529,[Textile exporter],4.5,6,None,None,None,Open now,"[0x80c2c624136ea88b:0xb0315367ed448771, 0x80c2...",https://www.google.com/maps/place//data=!4m2!3...


In [8]:
# Resumen de las features disponibles
df.columns

Index(['name', 'address', 'gmap_id', 'description', 'latitude', 'longitude',
       'category', 'avg_rating', 'num_of_reviews', 'price', 'hours', 'MISC',
       'state', 'relative_results', 'url'],
      dtype='object')

In [14]:
# Tamaño del dataframe
df.shape

(275001, 15)

In [13]:
# Hay valores nulos?
df.isna().sum()

name                     7
address              10062
gmap_id                  0
description         261846
latitude                 0
longitude                0
category              2261
avg_rating               0
num_of_reviews           0
price               261551
hours                82553
MISC                 80029
state                79478
relative_results     36230
url                      0
dtype: int64

Hay features que en su mayoría tienen NaN por lo que no aportan a una estadística o sistema de recomendación ("description" y "price"). Vamos a analizar las demás.

In [12]:
df.head()["address"].values

array(['Porter Pharmacy, 129 N Second St, Cochran, GA 31014',
       'City Textile, 3001 E Pico Blvd, Los Angeles, CA 90023',
       'San Soo Dang, 761 S Vermont Ave, Los Angeles, CA 90005',
       'Nova Fabrics, 2200 E 11th St, Los Angeles, CA 90021',
       'Nobel Textile Co, 719 E 9th St, Los Angeles, CA 90021'],
      dtype=object)

La columna "address" contiene valores separados por coma, vamos a tener que separar la info.
Se ve primero el nombre del comercio, luego dirección, ciudad y ZIP Code (state + 5 digitos) 

In [41]:
df.astype(str).duplicated().sum()

26573

Tenemos 26573 registros duplicados, debemos limpiarlos 

In [22]:
df.head(10)["category"]

0                                     [Pharmacy]
1                             [Textile exporter]
2                            [Korean restaurant]
3                                 [Fabric store]
4                                 [Fabric store]
5                                 [Fabric store]
6                                   [Restaurant]
7      [Nail salon, Waxing hair removal service]
8               [Bakery, Health food restaurant]
9    [Greeting card shop, Service establishment]
Name: category, dtype: object

In [21]:
df["category"].explode().nunique()

3769

La columna categoría es multilabel

In [15]:
df.head()["state"]

0    Open ⋅ Closes 6PM
1             Open now
2    Open ⋅ Closes 6PM
3    Open ⋅ Closes 5PM
4    Open ⋅ Closes 5PM
Name: state, dtype: object

La columna state no nos sirve, es un valor instantaneo sobre si el comercio está abierto o no.

In [25]:
df.head(1)["relative_results"].values

array([list(['0x88f16e41929435cf:0x5b2532a2885e9ef6', '0x88f16c32716531c1:0x5f19bdaa5044e4fa', '0x88f16e6e3f4a21df:0xcf495da9bb4d89ea'])],
      dtype=object)

Queda por ver si los relative results son comercios asociados. En este dataset no lo encontramos, deberíamos unirlos todos

Ahora pasamos a hacer una función que procese todos los json de manera automática

In [1]:
# Función para ejecutar los cambios vistos y unir los datasets de metadata-sitios

def cargar_archivos_json_2(ruta_base):

    # Variables para llevar el conteo de registros y nulos

    conteo_registros = 0
    conteo_duplicados = 0

    # Lista para almacenar los df que corresponden a cada json
    lista_dataframes = []

    # Recorre todas las carpetas dentro de la ruta_base
    for json_file in os.listdir(ruta_base):

        if json_file.endswith('.json'):
            json_path = os.path.join(ruta_base, json_file)

            # Lee el archivo JSON y carga los datos en un DataFrame
            df = pd.read_json(json_path, lines = True)

            # Borramos las columnas "description", "price" y "state"
            df = df.drop(columns = ["description", "price", "state"])

            # Hacemos el conteo de registros
            conteo_registros += df.shape[0]
            conteo_duplicados += df.astype(str).duplicated().sum()

            # Borramos registros duplicados
            df = df.astype(str).drop_duplicates()

            # Hacemos la transformación de la columna "address"
            # Creamos nuevas columnas separando el contenido (la primer columna es el "name" del lugar, ya lo tenemos)
            df[["address", "city", "state zip"]] = df["address"].str.split(', ', n = 3, expand=True).iloc[:, 1:]

            # Dividimos la columna state zip en dos columnas separando el estado y el código postal
            df[["state", "zip_code"]] = df["state zip"].str.extract('(.+?)\s(\d{5})')

            # Eliminamos las columnas innecesarias, incluyendo "direcciones" y "State Zip"
            df = df.drop(["state zip"], axis = 1)

            # Anexamos los dataframes
            lista_dataframes.append(df)

    # Concatena los dataFrames en uno solo
    metadata_sitios = pd.concat(lista_dataframes, ignore_index = True)

    return metadata_sitios, conteo_registros, conteo_duplicados

In [3]:
# Ruta base donde se encuentran las carpetas "review-estados"
ruta_base = "Datasets/Google Maps/metadata-sitios"
ruta_destino = "Datasets/Google Maps/metadata-sitios-clean"

# Llama a la función y almacena el conteo de nulos
metadata_sitios, conteo_registros, conteo_duplicados = cargar_archivos_json_2(ruta_base)

In [5]:
metadata_sitios.shape

(2998438, 15)

In [7]:
# Registros totales y nulos:
print(f"Registros totales: {conteo_registros}")
print(f"Registros duplicados: {conteo_duplicados}")

Registros totales: 3025011
Registros duplicados: 26573


In [8]:
# Vemos que el dataset tiene la información relevante y detallada, con la división que necesitabamos.
metadata_sitios.head(2)

,name,address,gmap_id,latitude,longitude,category,avg_rating,num_of_reviews,hours,MISC,relative_results,url,city,state,zip code
0,Porter Pharmacy,129 N Second St,0x88f16e41928ff687:0x883dad4fd048e8f8,32.3883,-83.3571,['Pharmacy'],4.9,16,"[['Friday', '8AM–6PM'], ['Saturday', '8AM–12PM...","{'Service options': ['In-store shopping', 'Sam...","['0x88f16e41929435cf:0x5b2532a2885e9ef6', '0x8...",https://www.google.com/maps/place//data=!4m2!3...,Cochran,GA,31014
1,City Textile,3001 E Pico Blvd,0x80c2c98c0e3c16fd:0x29ec8a728764fdf9,34.0188913,-118.2152898,['Textile exporter'],4.5,6,None,None,"['0x80c2c624136ea88b:0xb0315367ed448771', '0x8...",https://www.google.com/maps/place//data=!4m2!3...,Los Angeles,CA,90023


In [15]:
# Cuantós locales corresponden a cada estado?
metadata_sitios["state"].value_counts().head(52)

CA             269268
TX             234739
FL             190450
NY             145446
PA             102802
IL              93561
OH              90949
GA              85448
NC              85375
MI              81875
NJ              68693
VA              62106
WA              61310
TN              57686
IN              52987
AZ              52740
CO              52645
MO              51362
MA              49277
WI              48374
MN              43280
SC              42906
MD              40646
OR              39918
AL              39866
OK              36159
LA              34632
KY              33604
UT              28348
CT              27028
IA              26852
AR              25746
KS              24806
NV              23676
MS              19941
NM              17616
ID              17518
NE              16559
ME              13194
NH              12989
WV              12850
MT              11435
HI              10129
RI               8331
DE               7579
SD        

In [16]:
# Tenemos un conflicto con algunos registros, no respeta la división la columna "address". Debemos solucionar
metadata_sitios[metadata_sitios["state"] == "Houston, TX"]

,name,address,gmap_id,latitude,longitude,category,avg_rating,num_of_reviews,hours,MISC,relative_results,url,city,state,zip_code
1262,Beta Repair,We Come To You,0x8640a53845ef421d:0x3b1786d8195d380f,29.852546999999998,-95.189112,['Electronics store'],4.6,8,"[['Thursday', '11AM–7PM'], ['Friday', '11AM–7P...",{'Offerings': ['Repair services']},"['0x8640bb9ee367db27:0xc461561f621f1b8', '0x86...",https://www.google.com/maps/place//data=!4m2!3...,2200,"Houston, TX",77044
3506,"Patricia Choy, MD",MD,0x86409d0bd8bcaabf:0xd9448bec396ae128,29.5488909,-95.08596229999999,"['Obstetrician-gynecologist', ""Women's health ...",3.4,7,"[['Wednesday', '8AM–5PM'], ['Thursday', '8AM–5...","{'Service options': ['Online care'], 'Accessib...","['0x86409c8c1a90aa0b:0x9571c96019b4badf', '0x8...",https://www.google.com/maps/place//data=!4m2!3...,2060 Space Park Dr Suite 410,"Houston, TX",77058
4133,Preet BANQUET HALL C-25,14207 Tomball Parkway Hwy249@ Hall-25,0x8640ceea26935963:0x9e753c785d0bbdf8,29.923682999999997,-95.50035969999999,['Banquet hall'],3.4,15,None,"{'Service options': ['Delivery'], 'Accessibili...","['0x8640cfab7d1ac04f:0x610698e3f360bbfe', '0x8...",https://www.google.com/maps/place//data=!4m2!3...,Fallbrook Dr,"Houston, TX",77086
6471,"The Pearl Dermatology, PLLC",PLLC,0x8640c1a72240d08f:0x905adab96ecc95a1,29.7544425,-95.4534149,['Dermatologist'],5.0,86,"[['Wednesday', '8AM–5PM'], ['Thursday', '8AM–5...",{'Accessibility': ['Wheelchair accessible entr...,"['0x8640c393878ae88b:0xd6b6655cb494a618', '0x8...",https://www.google.com/maps/place//data=!4m2!3...,550 Post Oak Blvd #550,"Houston, TX",77027
8032,"Dentistry of Highland Village of Houston, TX",TX,0x8640c104a081c43d:0xc870b7fb973b390b,29.733991,-95.441096,"['Dentist', 'Cosmetic dentist', 'Dental clinic...",4.6,318,"[['Wednesday', '7AM–5PM'], ['Thursday', '7AM–5...",{'Accessibility': ['Wheelchair accessible entr...,"['0x8640c0fba1490221:0xe0c73e5aaf54e270', '0x8...",https://www.google.com/maps/place//data=!4m2!3...,3651 Weslayan St #208,"Houston, TX",77027
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2989742,"DermaTouch RN - Botox, Fillers and CoolSculpting",Fillers and CoolSculpting,0x8640cdf2a4f9d483:0x3fbb08564c04d711,29.976114799999998,-95.54835369999999,"['Medical spa', 'Facial spa', 'Health spa', 'L...",4.8,828,"[['Saturday', '9AM–2PM'], ['Sunday', 'Closed']...",{'From the business': ['Identifies as women-le...,"['0x8640d28831885e1b:0xda001db93e4f6a6f', '0x8...",https://www.google.com/maps/place//data=!4m2!3...,13725 Falba Rd,"Houston, TX",77070
2990256,"Michael Byrd, MD",MD,0x8640c1d5d05b9c17:0xed33c9afdc8e1973,29.7048639,-95.4447766,"['Doctor', 'Otolaryngologist']",4.9,11,"[['Saturday', 'Closed'], ['Sunday', 'Closed'],...",{'Accessibility': ['Wheelchair accessible entr...,None,https://www.google.com/maps/place//data=!4m2!3...,4191 Bellaire Blvd Suite 200,"Houston, TX",77025
2995809,Kayla Nail & Spa,Rankin Plaza,0x8640ca0666ae5603:0xf963045ed959edca,29.9655928,-95.3986437,"['Beauty salon', 'Nail salon']",3.2,79,"[['Saturday', '9AM–7PM'], ['Sunday', 'Closed']...",{'Accessibility': ['Wheelchair accessible entr...,"['0x8640ca054bc53223:0xd570119435f882d5', '0x8...",https://www.google.com/maps/place//data=!4m2!3...,802 Rankin Rd,"Houston, TX",77073
2998382,"David M. Bloome, MD",MD,0x8640c013661b6cdd:0xbbe5e7e006fd0e90,29.7003904,-95.40892629999999,"['Orthopedic surgeon', 'Doctor']",4.8,68,"[['Saturday', 'Closed'], ['Sunday', 'Closed'],...",{'Planning': ['Appointments recommended']},"['0x8640c070db205a1d:0x2cfbac2e3709182d', '0x8...",https://www.google.com/maps/place//data=!4m2!3...,7401 Main St,"Houston, TX",77030
